In [2]:
from os import listdir

file_path_1 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
file_path_2 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set2/"
file_path_3 = "C:/Users/Leste/OneDrive - Johns Hopkins/Desktop/BDD data/extracted/testing-RiskFactors-Complete/"
names_1 = [f for f in listdir(file_path_1) if f.endswith('.xml')]
names_2 = [f for f in listdir(file_path_2) if f.endswith('.xml')]
names_3 = [f for f in listdir(file_path_3) if f.endswith('.xml')]

In [3]:
import pandas as pd
import numpy as np

all1_df = pd.DataFrame(np.zeros((len(names_1), 3), dtype=object), columns=['text', 'annotation','loc'])
all2_df = pd.DataFrame(np.zeros((len(names_2), 3), dtype=object), columns=['text', 'annotation','loc'])
test_df = pd.DataFrame(np.zeros((len(names_3), 3), dtype=object), columns=['text', 'annotation','loc'])

In [4]:
import re
import xml.etree.ElementTree as ET


def to_df(names,df, file_path,PHI_status = True):
    #get text
    n = 0
    for name in names:
        tree = ET.parse(file_path + name)
        root = tree.getroot()

        ## Get the text
        nt = re.sub('\n',' ',root[0].text)
        nt = re.sub('\t',' ',nt) 
        nt = re.sub('"',"'",nt)
        ## sample 214 has a weird character
        nt = re.sub('>','&gt;',nt) 
        nt = re.sub('<','&lt;',nt)
        ## new wired character
        nt = re.sub('Â','',nt)
        nt = re.sub('â','',nt)
        nt = re.sub('€','',nt)
        nt = re.sub('™','',nt)
        df['text'][n] = nt
        n+=1
    
    #get annotations
    n = 0
    for name in names:
        tree = ET.parse(file_path + name)
        root = tree.getroot()
        ## Get the labels

        tag_list = []
        loc_list = []
        # get PHI labels if there are any
        if PHI_status == True:
            PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
            for k in range(PHI,len(root[1])):
                tag_list.append((root[1][k].attrib['text'],root[1][k].tag))
        
        # get the rest of labels
        for k in range(len(root[1])):
            for m in range(len(root[1][k])):
                if root[1][k][m].attrib.keys().__contains__('text') == False:
                    continue
                if root[1][k][m].attrib['text'] == 'Record':
                    continue
                tag_list.append((root[1][k][m].attrib['text'],root[1][k][m].tag))
                loc_list.append((root[1][k][m].attrib['start'],root[1][k][m].attrib['end']))
        df['annotation'][n] = tag_list
        df['loc'][n] = loc_list
        n+=1
    return df

all_1 = to_df(names_1,all1_df, file_path_1, PHI_status = False)
all_2 = to_df(names_2,all2_df, file_path_2, PHI_status = False)
test_df = to_df(names_3,test_df, file_path_3, PHI_status = False)
train_df = pd.concat([all_1, all_2], ignore_index=True)

In [9]:
def intersection(ls1, ls2):
    index = [i for i, x in enumerate([x == ls2[0] for x in ls1]) if x]
    cut = 1000
    for i in index:
        eva = sum([ls1[x+i] != ls2[x] for x in range(len(ls1)-i)])
        if eva < cut:
            cut = i
    out = "".join(ls1[0:cut]+ls2)
    return out

def rm_dup(df):
    symbol = [",",".","-"]
    for i in range(df.shape[0]):

        for j in range(len(df['annotation'][i])):

            for k in np.arange(j+1,len(df['annotation'][i])):
                # move on if the compared lables are ("","")
                if df['loc'][i][j] == ('',''):
                    break
                if df['loc'][i][k] == ('',''):
                    continue

                ## preprocess tagged text and location
                # remove extra spaces in the beginning and end of the annotation
                if re.search("^ +.*",df['annotation'][i][j][0]) != None or re.search(".* +$",df['annotation'][i][j][0]) != None:
                    front = len(df['annotation'][i][j][0]) - len(re.sub("^ +","",df['annotation'][i][j][0]))
                    end = len(df['annotation'][i][j][0]) - len(re.sub(" +$","",df['annotation'][i][j][0]))
                    df['loc'][i][j] = (str(int(df['loc'][i][j][0])+front),str(int(df['loc'][i][j][1])-end))
                    df['annotation'][i][j] = (re.sub(" +$","", re.sub("^ +","",df['annotation'][i][j][0])),df['annotation'][i][j][1])

                if re.search("^ +.*",df['annotation'][i][k][0]) != None or re.search(".* +$",df['annotation'][i][k][0]) != None:
                    front = len(df['annotation'][i][k][0]) - len(re.sub("^ +","",df['annotation'][i][k][0]))
                    end = len(df['annotation'][i][k][0]) - len(re.sub(" +$","",df['annotation'][i][k][0]))
                    df['loc'][i][k] = (str(int(df['loc'][i][k][0])+front),str(int(df['loc'][i][k][1])-end))
                    df['annotation'][i][k] = (re.sub(" +$","", re.sub("^ +","",df['annotation'][i][k][0])),df['annotation'][i][k][1])

                # find location contained within each other
                if int(df['loc'][i][j][0]) >= int(df['loc'][i][k][0]) and int(df['loc'][i][j][1]) <= int(df['loc'][i][k][1]):
                    df['loc'][i][j] = ('','')
                    df['annotation'][i][j] = ('','')
                    continue
                
                elif int(df['loc'][i][j][0]) <= int(df['loc'][i][k][0]) and int(df['loc'][i][j][1]) >= int(df['loc'][i][k][1]):
                    df['loc'][i][k] = ('','')
                    df['annotation'][i][k] = ('','')
                    continue

                # find location that overlap
                if int(df['loc'][i][j][0]) < int(df['loc'][i][k][0]) and int(df['loc'][i][j][1]) < int(df['loc'][i][k][1]) and int(df['loc'][i][j][1]) > int(df['loc'][i][k][0]):
                    #print(i,j,k)
                    new_s = intersection(df['annotation'][i][j][0].split(),df['annotation'][i][k][0].split())
                    df['annotation'][i][j] = (new_s,df['annotation'][i][j][1])
                    df['loc'][i][j] = (df['loc'][i][j][0],df['loc'][i][k][1])
                    df['loc'][i][k] = ('','')
                    df['annotation'][i][k] = ('','')
                    continue
            
                elif int(df['loc'][i][j][0]) > int(df['loc'][i][k][0]) and int(df['loc'][i][j][1]) > int(df['loc'][i][k][1]) and int(df['loc'][i][j][0]) < int(df['loc'][i][k][1]):
                    #print(i,j,k)
                    #the order here matters, the first one should be the left most
                    new_s = intersection(df['annotation'][i][k][0].split(),df['annotation'][i][j][0].split())
                    df['annotation'][i][k] = (new_s,df['annotation'][i][k][1])
                    df['loc'][i][k] = (df['loc'][i][k][0],df['loc'][i][j][1])
                    df['loc'][i][j] = ('','')
                    df['annotation'][i][j] = ('','')
                    continue

    return df

In [10]:
l1 = "apple is red"
l2 = "apple is red and green"
intersection(l1,l2)

'apple is red and green'

# old code used to clean text, replaced by merge two sentence together

In [5]:
                ## preprocess tagged text and location
                # remove extra spaces in the beginning and end of the annotation
                if re.search("^ +.*",df['annotation'][i][j][0]) != None or re.search(".* +$",df['annotation'][i][j][0]) != None:
                    front = len(df['annotation'][i][j][0]) - len(re.sub("^ +","",df['annotation'][i][j][0]))
                    end = len(df['annotation'][i][j][0]) - len(re.sub(" +$","",df['annotation'][i][j][0]))
                    df['loc'][i][j] = (str(int(df['loc'][i][j][0])+front),str(int(df['loc'][i][j][1])-end))
                    df['annotation'][i][j] = (re.sub(" +$","", re.sub("^ +","",df['annotation'][i][j][0])),df['annotation'][i][j][1])

                if re.search("^ +.*",df['annotation'][i][k][0]) != None or re.search(".* +$",df['annotation'][i][k][0]) != None:
                    front = len(df['annotation'][i][k][0]) - len(re.sub("^ +","",df['annotation'][i][k][0]))
                    end = len(df['annotation'][i][k][0]) - len(re.sub(" +$","",df['annotation'][i][k][0]))
                    df['loc'][i][k] = (str(int(df['loc'][i][k][0])+front),str(int(df['loc'][i][k][1])-end))
                    df['annotation'][i][k] = (re.sub(" +$","", re.sub("^ +","",df['annotation'][i][k][0])),df['annotation'][i][k][1])

                # remove extra punctuation in the end of the annotation
                if sum([df['annotation'][i][j][0][-1] == f for f in symbol]) > 0:
                    df['annotation'][i][j] = (df['annotation'][i][j][0][:-1],df['annotation'][i][j][1])
                    df['loc'][i][j] = (df['loc'][i][j][0],str(int(df['loc'][i][j][1])-1))
                    
                if sum([df['annotation'][i][j][0][-1] == f for f in symbol]) > 0:
                    df['annotation'][i][k] = (df['annotation'][i][k][0][:-1],df['annotation'][i][k][1])
                    df['loc'][i][k] = (df['loc'][i][k][0],str(int(df['loc'][i][k][1])-1))

                if sum([df['annotation'][i][j][0][0] == f for f in symbol]) > 0:
                    df['annotation'][i][j] = (df['annotation'][i][j][0][1:],df['annotation'][i][j][1])
                    df['loc'][i][j] = (str(int(df['loc'][i][j][0])+1),df['loc'][i][j][1])
            
                if sum([df['annotation'][i][k][0][0] == f for f in symbol]) > 0:
                    df['annotation'][i][k] = (df['annotation'][i][k][0][1:],df['annotation'][i][k][1])
                    df['loc'][i][k] = (str(int(df['loc'][i][k][0])+1),df['loc'][i][k][1])

NameError: name 're' is not defined

In [6]:
train_n = rm_dup(train_df)
#test_n = rm_dup(test)


In [ ]:
for j in range(train_n.shape[0]):
    train_n['annotation'][j] = [train_n['annotation'][j][i] for i in range(len(train_n['annotation'][j])) if train_n['annotation'][j][i][0] != '']
    train_n['loc'][j] = [train_n['loc'][j][i] for i in range(len(train_n['loc'][j])) if train_n['loc'][j][i][0] != '']
    start = np.array([int(train_n['loc'][j][i][0]) for i in range(len(train_n['loc'][j]))])
    order = start.argsort()
    train_n['annotation'][j] = [train_n['annotation'][j][i] for i in order]
    train_n['loc'][j] = [train_n['loc'][j][i] for i in order]
train = train_n.drop('loc', axis=1)

In [ ]:

for j in range(test_n.shape[0]):
    test_n['annotation'][j] = [test_n['annotation'][j][i] for i in range(len(test_n['annotation'][j])) if test_n['annotation'][j][i][0] != '']
    test_n['loc'][j] = [test_n['loc'][j][i] for i in range(len(test_n['loc'][j])) if test_n['loc'][j][i][0] != '']
    start = np.array([int(test_n['loc'][j][i][0]) for i in range(len(test_n['loc'][j]))])
    order = start.argsort()
    test_n['annotation'][j] = [test_n['annotation'][j][i] for i in order]
    test_n['loc'][j] = [test_n['loc'][j][i] for i in order]
test = test_n.drop('loc', axis=1)

# modify testing text (change coronary arterary disease to CAD ...)

In [ ]:
test['text'] = test['text'].apply(lambda x: re.sub('coronary artery disease','cad',x))
test['text'] = test['text'].apply(lambda x: re.sub('Blood Pressure','bp',x))
test['text'] = test['text'].apply(lambda x: re.sub('blood pressure','bp',x))
test['text'] = test['text'].apply(lambda x: re.sub('Blood pressure','bp',x))
test['text'] = test['text'].apply(lambda x: re.sub('blood Pressure','bp',x))
test['text'] = test['text'].apply(lambda x: re.sub('Blood pressure','bp',x))
test['text'] = test['text'].apply(lambda x: re.sub('&#8211','',x))
#test['text'] = test['text'].apply(lambda x: re.sub(' p\.o\. ','per oral',x))
#test['text'] = test['text'].apply(lambda x: re.sub(' h/o ','had',x))
#test['text'] = test['text'].apply(lambda x: x.lower())

In [ ]:
## word_tokenize() seperate (lisinopril) into (, lis..., )
word_tokens = train['annotation'][0][2][0].strip().split()
filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in sw]
filtered_sentence

In [ ]:
import nltk
from nltk.corpus import stopwords
sw = stopwords.words('english')
sw.remove("no", "nor", "not","don't","didn't","doesn't","isn't","aren't","wasn't","weren't","haven't","hasn't","hadn't","won't","wouldn't","shouldn't","can't","couldn't","mustn","mustn't","mightn't","mightn't","needn't","needn't","oughtn't","shan't","shan't","shouldn't","wasn't","weren't","won't","wouldn't","t")
for i in range(test.shape[0]):
    for x in range(len(test['annotation'][i])):
        word_tokens = test['annotation'][i][x][0].strip().split()
        # converts the words in word_tokens to lower case and then checks whether 
        #they are present in stop_words or not
        filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in sw]
        tagged_things = ' '.join(filtered_sentence)
        tagged_things = re.sub('coronary artery disease','cad',tagged_things)
        tagged_things = re.sub('Blood Pressure','bp',tagged_things)
        tagged_things = re.sub('blood pressure','bp',tagged_things)
        tagged_things = re.sub('Blood pressure','bp',tagged_things)
        tagged_things = re.sub('blood Pressure','bp',tagged_things)
        tagged_things = re.sub('&#8211','',tagged_things)
        #tagged_things = re.sub(' p\.o\. ',' per oral ',tagged_things)
        #tagged_things = re.sub(' h/o ','had',tagged_things)
        test['annotation'][i][x] = (tagged_things,test['annotation'][i][x][1])

In [ ]:
train['text'] = train['text'].apply(lambda x: re.sub('coronary artery disease','cad',x))
train['text'] = train['text'].apply(lambda x: re.sub('Blood Pressure','bp',x))
train['text'] = train['text'].apply(lambda x: re.sub('blood pressure','bp',x))
train['text'] = train['text'].apply(lambda x: re.sub('Blood pressure','bp',x))
train['text'] = train['text'].apply(lambda x: re.sub('blood Pressure','bp',x))
train['text'] = train['text'].apply(lambda x: re.sub('Blood pressure','bp',x))
train['text'] = train['text'].apply(lambda x: re.sub('&#8211','',x))
#train['text'] = train['text'].apply(lambda x: re.sub(' p\.o\. ',' per oral ',x))
#train['text'] = train['text'].apply(lambda x: re.sub(' h/o ',' had ',x))
#train['text'] = train['text'].apply(lambda x: x.lower())

In [ ]:

for i in range(train.shape[0]):
    for x in range(len(train['annotation'][i])):
        word_tokens = train['annotation'][i][x][0].strip().split()
        # converts the words in word_tokens to lower case and then checks whether 
        #they are present in stop_words or not
        filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in sw]
        tagged_things = ' '.join(filtered_sentence)
        tagged_things = re.sub('coronary artery disease','cad',tagged_things)
        tagged_things = re.sub('Blood Pressure','bp',tagged_things)
        tagged_things = re.sub('blood pressure','bp',tagged_things)
        tagged_things = re.sub('Blood pressure','bp',tagged_things)
        tagged_things = re.sub('blood Pressure','bp',tagged_things)
        tagged_things = re.sub('&#8211','',tagged_things)
        #tagged_things = re.sub(' p\.o\. ',' per oral ',tagged_things)
        #tagged_things = re.sub(' h/o ',' had ',tagged_things)
        train['annotation'][i][x] = (tagged_things,train['annotation'][i][x][1])


In [ ]:
# remove stop words in text to match tags
for i in range(train.shape[0]):
    word_tokens = train['text'][i].strip().split()
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in sw]
    train['text'][i] = ' '.join(filtered_sentence)

for i in range(test.shape[0]):
    word_tokens = train['text'][i].strip().split()
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in sw]
    test['text'][i] = ' '.join(filtered_sentence)

In [ ]:
print(sw)
sw.remove("no", "nor", "not","don't","didn't","doesn't","isn't","aren't","wasn't","weren't","haven't","hasn't","hadn't","won't","wouldn't","shouldn't","can't","couldn't","mustn","mustn't","mightn't","mightn't","needn't","needn't","oughtn't","shan't","shan't","shouldn't","wasn't","weren't","won't","wouldn't","t")

In [ ]:
from tqdm import tqdm
from difflib import SequenceMatcher
import pickle

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)

    return match_list, string


def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = e_type
        else:
            word_dict.loc[index,'label'] = e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)
            
    return text

def to_txt(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            text_ = text    
            print(df.index[df['text']== text_].tolist())    
            match_list = []
            for i in annotation:
                a,text_= matcher(text_, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text_, match_list)

            for i in range(d.shape[0]):
                f.writelines(d['word'][i] + ' ' + d['label'][i] +'\n')
            f.writelines('\n')
            
def main(input,save_path):

    data = input
    to_txt(data, save_path)
    
#if __name__ == '__main__':
path = 'C:/Users/Leste/OneDrive - Johns Hopkins/Documents/GitHub/nlpsumm/BERT/data_processed/'

#main(train,path+'train.txt')
main(train,path+'train.txt')
#main(test,path+'test_clean(no_BIO).txt')

In [ ]:
print(train_n['annotation'][[26]].tolist())
print(train_n['loc'][[26]].tolist())


In [ ]:
a = "repeat cath in 12/94 which revealed in stent restenosis of prior LAD stent"
b = "repeat cath in 12/94 which revealed in stent restenosis of prior LAD stent."


In [ ]:
df = train.copy()
annotation = df.annotation[26]
text_ = df.text[26]

match_list = []
for i in annotation:
    print(i)
    a, text_ = matcher(text_, i[0])
    match_list.append((a[0][0], a[0][1], i[1]))
    d = create_labs(text_, match_list)


In [ ]:
text_

In [ ]:
a = "Today he arrived via SDA for cath that showed his native disease to be severe with a 70% stenosis"
b = "cath that showed his native disease to be severe with a 70% stenosis involving the ostium of the first diagonal branch. The mid-distal LAD is small and tapering with diffuse disease  up to 80%. The first diagonal has a high grade ostial stenosis. LCX has 40-50% stenosis proximal to the first OM. RCA has 80% stenosis proximally with competitive flow from the SVG. The LIMA-&gt; D1 is patent and the SVG-&gt; RCA is patent. The SVG-&gt; LAD is not visualized"
s = [a,b]
new_s = sorted(s, key=lambda x:s[0].index(x[0]))
a = new_s[0]
b = new_s[-1]
final_s = a[:a.index(b[0])]+b
final_s

In [ ]:
text

In [ ]:
# checking p.o. in training labels
for i in range(test.shape[0]):
    for x in range(len(train['annotation'][i])):
        y=re.search("&#8211",train['annotation'][i][x][0])
        if y:
            print(train['annotation'][i][x][0])
            print(i)


In [ ]:

for i in range(test.shape[0]):
    x = re.search("&#8211",test['text'][i])
    if x:
        print(x)
        print(i)

In [ ]:
names_3[513]